## Publish a Downhole Collection object

This example shows how to convert drilling data in CSV format into an Evo geoscience object using the Evo Python SDK.

### Requirements

You must have a Seequent account with the Evo entitlement to use this notebook.

The following parameters must be provided:

- The client ID of your Evo application.
- The callback/redirect URL of your Evo application.

To obtain these app credentials, refer to the [Apps and tokens guide](https://developer.seequent.com/docs/guides/getting-started/apps-and-tokens) in the Seequent Developer Portal.

In [ ]:
import uuid

import pandas as pd
from evo_schemas.components import (
    BoundingBox_V1_0_1,
    CategoryAttribute_V1_1_0,
    CategoryData_V1_0_1,
    ContinuousAttribute_V1_1_0,
    Crs_V1_0_1_EpsgCode,
    DistanceTable_V1_2_0_Distance,
    Intervals_V1_0_1,
    IntervalTable_V1_2_0_FromTo,
    NanCategorical_V1_0_1,
    NanContinuous_V1_0_1,
)
from evo_schemas.elements import (
    FloatArray1_V1_0_1,
    FloatArray2_V1_0_1,
    FloatArray3_V1_0_1,
    IntegerArray1_V1_0_1,
    LookupTable_V1_0_1,
)
from evo_schemas.objects import (
    DownholeCollection_V1_2_0,
    DownholeCollection_V1_2_0_Collections_DistanceTable,
    DownholeCollection_V1_2_0_Collections_IntervalTable,
    DownholeCollection_V1_2_0_Location,
    DownholeCollection_V1_2_0_Location_Holes,
    DownholeCollection_V1_2_0_Location_Path,
)
from IPython.display import HTML, display

from evo.notebooks import FeedbackWidget, ServiceManagerWidget
from evo.objects import ObjectAPIClient

cache_location = "data"
input_path = f"{cache_location}/input"

# Evo app credentials
client_id = "<client_id>"
redirect_url = "<redirect_url>"

manager = await ServiceManagerWidget.with_auth_code(
    discovery_url="https://discover.api.seequent.com",
    redirect_url=redirect_url,
    client_id=client_id,
    cache_location=cache_location,
).login()

### Use the Evo Python SDK to create an object client and a data client

In [ ]:
# The object client will manage your auth token and Geoscience Object API requests.
object_client = ObjectAPIClient(manager.get_environment(), manager.get_connector())

# The data client will manage saving your data as Parquet and publishing your data to Evo storage.
data_client = object_client.get_data_client(manager.cache)

### Define helper functions

These functions assist with assembling the elements and components of geoscience objects and for viewing the new object in the Evo portal.

In [ ]:
import numpy as np


def create_hole_id_mapping(hole_id_table, value_list):
    """
    Create a hole ID mapping table based on the hole ID table and the value list.

    Args:
        hole_id_table (pd.DataFrame): The hole ID lookup table.
        value_list (pd.DataFrame): The value list to create the mapping from.

    Returns:
        mapping_df (pd.DataFrame): The hole ID mapping table.
    """

    num_keys = len(hole_id_table.index)

    mapping_df = pd.DataFrame(list())
    mapping_df["hole_index"] = hole_id_table["key"]
    mapping_df["offset"] = [0] * num_keys
    mapping_df["count"] = [0] * num_keys

    mapping_df["hole_index"] = mapping_df["hole_index"].astype("int32")
    mapping_df["offset"] = mapping_df["offset"].astype("uint64")
    mapping_df["count"] = mapping_df["count"].astype("uint64")

    prev_value = ""
    key = ""
    count = 0
    offset = 0

    for index, row in value_list.iterrows():
        new_value = row["data"]

        if new_value != prev_value:
            if prev_value != "":
                mapping_df.loc[mapping_df["hole_index"] == key, "count"] = count
                mapping_df.loc[mapping_df["hole_index"] == key, "offset"] = offset
                offset += count

            mask = hole_id_table["value"] == new_value
            masked_df = hole_id_table[mask]
            try:
                key_row = masked_df.iloc[[0]]
            except IndexError:
                print("Ignoring this hole ID")
                continue

            key = key_row["key"].iloc[0]
            count = 1
            prev_value = new_value
        else:
            count += 1

    mapping_df.loc[mapping_df["hole_index"] == key, "count"] = count
    mapping_df.loc[mapping_df["hole_index"] == key, "offset"] = offset

    return mapping_df


def create_category_lookup_and_values(attribute):
    """
    Create a category lookup table and the associated column of mapped key values.

    Args:
        attribute (pd.DataFrame): An attribute of a geoscience object.

    Returns:
        table_df (pd.DataFrame): The category lookup table.
        values_df (pd.DataFrame): The associated column with mapped key values.
    """

    # Replace NaN with empty string
    attribute.replace(np.nan, "", regex=True, inplace=True)
    set_obj = set(attribute["data"])
    list_obj = list(set_obj)
    list_obj.sort()
    num_unique_elements = len(list_obj)

    # Create lookup table
    table_df = pd.DataFrame([])
    table_df["key"] = list(range(1, num_unique_elements + 1))
    table_df["value"] = list_obj

    # Create data column
    values_df = pd.DataFrame([])
    values_df["data"] = attribute["data"].map(table_df.set_index("value")["key"])
    return table_df, values_df


def build_portal_url(object_metadata):
    """
    Build the URL to view the object in the Evo Portal.

    Args:
        object_metadata (str): The object metadata object returned after the object is created.

    Returns:
        str: The URL to view the object in the Evo Portal.
    """

    hub_url = object_metadata.environment.hub_url
    hub_name = hub_url.split("://")[1].split(".")[0]
    org_id = object_metadata.environment.org_id
    workspace_id = object_metadata.environment.workspace_id
    object_id = object_metadata.id

    url = f"https://evo.seequent.com/{org_id}/workspaces/{hub_name}/{workspace_id}/viewer?id={object_id}"

    display(HTML(f'<a href="{url}" target="_blank">View object in the Evo Portal</a>'))

### Define object metadata

Geoscience object data must conform to a specific object schema. The `evo-schemas` package provides Pydantic models that make it easy to work with the equivalent JSON schemas. 
For this example we'll use v1.2.0 of the downhole-collection schema, via the relevant Pydantic model.

Enter values for these parameters that are required by the object schema.
- `object_hole_id`: The column name that represents your hole ID. This value should be the same across all input files.
- `object_name`: The name of the object.
- `object_path`: The file path where the object will be found.
- `object_epsg_code`: (Optional) The EPSG region code that matches the location of your data. Leave as `None` if not required.
- `object_tags`: (Optional) A dictionary of additional tags to be assigned to the object. Leave as `None` is not required.

In [ ]:
# Set the name of the hole ID parameter in the data.
object_hole_id = "BHID"

# Set other object properties.
object_name = "DHC_SDK_demo"
object_path = "Jupyter_Example"
object_epsg_code = 32650
object_tags = {"Source": "Jupyter Notebook"}

# Define a coordinate reference system (CRS) for the object.
coordinate_reference_system = Crs_V1_0_1_EpsgCode(epsg_code=object_epsg_code)

# Create an empty list to store geoscience object collections.
collections = []

# Define the object path.
full_obj_path = f"{object_path}/{object_name}.json"

### Define object attributes and keys

In [ ]:
# List all of the attributes to be included in the object. Every attribute must have a unique key associated with it.
# Keys must be unique across the entire object, and we recommend saving a reference to the keys for later use.
collar_attributes_ref = {
    "Hole Type": str(uuid.uuid4()),
    "Random Number": str(uuid.uuid4()),
}

survey_attributes_ref = {
    "Random Numeric": str(uuid.uuid4()),
}

object_attributes_ref = {
    "Wolfpass_WP_assay": {
        "CU_pct": str(uuid.uuid4()),
        "AU_gpt": str(uuid.uuid4()),
        "DENSITY": str(uuid.uuid4()),
    },
    "Wolfpass_WP_lith": {
        "ROCK": str(uuid.uuid4()),
        "grouped_lith": str(uuid.uuid4()),
        "Split_Dykes": str(uuid.uuid4()),
    },
}

depth_attribute_ref = {
    "Wolfpass_depths": {
        "ATTR": str(uuid.uuid4()),
    }
}

### Collar

In [ ]:
# Define input and output file paths.
input_file_path = f"{input_path}/Wolfpass_collar.csv"

# Load the collar file, count the number of hole IDs and sort the data based on the hole ID.
input_df = pd.read_csv(input_file_path)
num_hole_ids = len(input_df.index)
sorted_collar_df = input_df.sort_values([object_hole_id]).reset_index(drop=True)

sorted_collar_df.head()

### Hole ID table

Most components in our downhole collection object will reference the hole IDs defined in the collar table.

This means that we must create a 2-column dataframe that maps a unique `key` to a `hole ID`.

The `location` component of the downhole collection object makes use of this mapping, so we provide a 1-column dataframe that lists the keys in the order they are displayed in the input file.

In [ ]:
# Create a dataframe for the hole IDs.
hole_id_table_df = pd.DataFrame()
hole_id_table_df["key"] = [i for i in range(1, num_hole_ids + 1)]
hole_id_table_df["value"] = sorted_collar_df[object_hole_id]
hole_id_table_component = LookupTable_V1_0_1.from_dict(data_client.save_dataframe(hole_id_table_df))

### Hole ID values

In [ ]:
# Create a dataframe and generate a list from 1 to `n`, where `n` is the number of hole IDs.
# This table represents the list of hole IDs created in the previous step.
hole_id_values_df = pd.DataFrame()
hole_id_values_df["data"] = [i for i in range(1, num_hole_ids + 1)]
hole_id_values_component = IntegerArray1_V1_0_1.from_dict(data_client.save_dataframe(hole_id_values_df))

hole_id_component = CategoryData_V1_0_1(
    table=hole_id_table_component,
    values=hole_id_values_component,
)

### Coordinates

In [ ]:
# Create a dataframe and copy the required columns.
# NOTE: The columns must be renamed to `x`, `y` and `z`.
coordinates_df = pd.DataFrame()
coordinates_df[["x", "y", "z"]] = input_df[["XCOLLAR", "YCOLLAR", "ZCOLLAR"]]

# Create a `BoundingBox_V1_0_1` component for the bounding box.
bounding_box = BoundingBox_V1_0_1(
    min_x=coordinates_df["x"].min(),
    max_x=coordinates_df["x"].max(),
    min_y=coordinates_df["y"].min(),
    max_y=coordinates_df["y"].max(),
    min_z=coordinates_df["z"].min(),
    max_z=coordinates_df["z"].max(),
)

# Create a `FloatArray3_V1_0_1` element for the coordinates.
coordinates_component = FloatArray3_V1_0_1.from_dict(data_client.save_dataframe(coordinates_df))

In [ ]:
# Load the collar attributes

collar_attributes = []

for heading_name, heading_key in collar_attributes_ref.items():
    print(heading_name)
    if heading_name == "Random Number":
        # Construct a FloatArray1 component.
        values_df = pd.DataFrame()
        values_df = input_df.loc[:, [heading_name]].copy().astype(float).reset_index(drop=True)
        values = FloatArray1_V1_0_1.from_dict(data_client.save_dataframe(values_df))

        attribute = ContinuousAttribute_V1_1_0(
            name=heading_name, key=heading_key, nan_description=NanContinuous_V1_0_1(values=[]), values=values
        )

        collar_attributes.append(attribute)
    elif heading_name == "Hole Type":
        # Construct a LookupTable and IntegerArray1 component.
        data_df = input_df.loc[:, [heading_name]].copy().reset_index(drop=True)
        data_df.rename(columns={heading_name: "data"}, inplace=True)
        table_df, values_df = create_category_lookup_and_values(data_df)
        table = LookupTable_V1_0_1.from_dict(data_client.save_dataframe(table_df))
        values = IntegerArray1_V1_0_1.from_dict(data_client.save_dataframe(values_df.astype("int32")))

        attribute = CategoryAttribute_V1_1_0(
            name=heading_name,
            key=heading_key,
            nan_description=NanCategorical_V1_0_1(values=[]),
            table=table,
            values=values,
        )

        collar_attributes.append(attribute)

print(collar_attributes)

### Distances

In [ ]:
# Create a distances dataframe and copy the required columns.
# NOTE: The downhole collection object requires 3 columns: `final`, `target` and `current`,
# but in this data set there is only 1 depth value.
# To solve this problem we duplicate the depth value across the other 2 columns.
distances_df = pd.DataFrame()
distances_df["final"] = input_df["maxdepth"]
distances_df["target"] = distances_df.loc[:, "final"]
distances_df["current"] = distances_df.loc[:, "final"]

# Create a `FloatArray3_V1_0_1` element for the distances.
distances_component = FloatArray3_V1_0_1.from_dict(data_client.save_dataframe(distances_df))

### Survey

In [ ]:
# Define input and output file paths.
input_file_path = f"{input_path}/Wolfpass_survey.csv"

# Load the survey file.
df = pd.read_csv(input_file_path)
df = df.sort_values([object_hole_id]).reset_index(drop=True)
df.head()

### Path

In [ ]:
# Remove hole IDs from the survey file that don't appear in the hole ID table.
# If you leave these rogue hole IDs in your survey table Leapfrog won't be able to import your geoscience object.
df.drop(df[~df[object_hole_id].isin(list(hole_id_table_df["value"]))].index, inplace=True)
df.reset_index(drop=True)
df.head()

In [ ]:
# Create a dataframe and copy the required survey column.
survey_values_df = pd.DataFrame()
survey_values_df["data"] = df[object_hole_id]

In [ ]:
# Load the survey attributes

# Construct a FloatArray1 component for each attribute column in the survey table.
survey_attributes = []

for heading_name, heading_key in survey_attributes_ref.items():
    values_df = pd.DataFrame()
    values_df = df.loc[:, [heading_name]].copy().astype(float).reset_index(drop=True)
    values = FloatArray1_V1_0_1.from_dict(data_client.save_dataframe(values_df))

    attribute = ContinuousAttribute_V1_1_0(
        name=heading_name, key=heading_key, nan_description=NanContinuous_V1_0_1(values=[]), values=values
    )

    survey_attributes.append(attribute)

print(survey_attributes)

In [ ]:
# Rename the column names to what is expected in the geoscience object and drop the hold ID column.
df = df.rename(columns={"DIP": "dip", "AT": "distance", "BRG": "azimuth"})
# Drop the hole ID column and all columns listed in survey_attributes
cols_to_drop = [object_hole_id] + list(survey_attributes_ref.keys())
df = df.drop(columns=cols_to_drop, axis=1)
df.head()

In [ ]:
# Swap the column order to match what the geoscience object
# requires (distance/azimuth/dip) and save the result in a new dataframe.
# NOTE: If your data set already has the correct ordering you can skip this step.
path_df = df.iloc[:, [0, 2, 1]]
path_df.head()

In [ ]:
# Create a `DownholeCollection_V1_2_0_Location_Path` object to handle the *path* component.
path_component = DownholeCollection_V1_2_0_Location_Path.from_dict(data_client.save_dataframe(path_df))
path_component.attributes = survey_attributes

### Holes

In [ ]:
# Using the hole ID table from earlier, create a mapping between
# the hole IDs defined in the collar table and the ones found in the survey table.
holes_df = create_hole_id_mapping(hole_id_table=hole_id_table_df, value_list=survey_values_df)
holes_df.head()

In [ ]:
# Create a `DownholeCollection_V1_2_0_Location_Holes` object to handle the *holes* component.
holes_component = DownholeCollection_V1_2_0_Location_Holes.from_dict(data_client.save_dataframe(holes_df))

In [ ]:
# Create a `DownholeCollection_V1_2_0_Location` object which combines all
# of the components created so far in this section.
location_component = DownholeCollection_V1_2_0_Location(
    distances=distances_component,
    holes=holes_component,
    hole_id=hole_id_component,
    path=path_component,
    coordinates=coordinates_component,
    attributes=collar_attributes,
)

### Distance table

In [ ]:
# Enter the name of the input csv file (without the file extension) in the `collection_name` variable.
# The same name will be applied to this collection in the new geoscience object, but this is configurable.
collection_name = "Wolfpass_depths"
input_file_path = f"{input_path}/{collection_name}.csv"

# Load the assay file and find the length.
df = pd.read_csv(input_file_path)
orig_count = len(df)
df.head()

In [ ]:
# Create an **depth_bhid_values** dataframe and copy the hole IDs.
depth_bhid_values_df = pd.DataFrame()
depth_bhid_values_df["data"] = df[object_hole_id]
depth_bhid_values_df.sort_values(by="data", inplace=True)
depth_bhid_values_df.reset_index(drop=True, inplace=True)
depth_bhid_values_df.head()

#### Depths

In [ ]:
# Create a dataframe and copy the *DEPTH* column to it.
depth_interval_table_df = df.loc[:, ["DEPTH"]].copy().reset_index(drop=True)
depth_interval_table_df.columns = ["data"]
depth_interval_table_df.head()

In [ ]:
# Create a component that represents the depths.
depths = FloatArray1_V1_0_1.from_dict(data_client.save_dataframe(depth_interval_table_df))

#### Holes

In [ ]:
# Create a mapping between the hole IDs in the depth table and the hole IDs from the collar table.
depth_holes_df = create_hole_id_mapping(hole_id_table=hole_id_table_df, value_list=depth_bhid_values_df)
depth_holes_df.head()

In [ ]:
# Create a component that represents the depth holes.
location_holes_component = DownholeCollection_V1_2_0_Location_Holes.from_dict(
    data_client.save_dataframe(depth_holes_df)
)

#### Depth Attributes

In [ ]:
# Construct a FloatArray1 component for each column in the assay table.
attributes = []

for heading_name, heading_key in depth_attribute_ref["Wolfpass_depths"].items():
    values_df = pd.DataFrame()
    values_df = df.loc[:, [heading_name]].copy().astype(float).reset_index(drop=True)
    values = FloatArray1_V1_0_1.from_dict(data_client.save_dataframe(values_df))

    attribute = ContinuousAttribute_V1_1_0(
        name=heading_name, key=heading_key, nan_description=NanContinuous_V1_0_1(values=[]), values=values
    )

    attributes.append(attribute)

In [ ]:
# Create the `DownholeCollection_V1_2_0_Collections_DistanceTable` object by combining the *holes* and *distance* components.
collection = DownholeCollection_V1_2_0_Collections_DistanceTable(
    name=collection_name,
    holes=location_holes_component,
    distance=DistanceTable_V1_2_0_Distance(values=depths, attributes=attributes),
)

collections.append(collection)

### Assay

In [ ]:
# Enter the name of the input csv file (without the file extension) in the `collection_name` variable.
# The same name will be applied to this collection in the new geoscience object, but this is configurable.
collection_name = "Wolfpass_WP_assay"
input_file_path = f"{input_path}/{collection_name}.csv"

# Load the assay file and find the length.
df = pd.read_csv(input_file_path)
orig_count = len(df)
df.head()

In [ ]:
# Remove hole IDs from the survey file that don't appear in the hole ID table.
# If you leave these rogue hole IDs in your survey table Leapfrog won't be able to import your geoscience object.
df.drop(df[~df[object_hole_id].isin(list(hole_id_table_df["value"]))].index, inplace=True)
df.reset_index(drop=True)
new_count = len(df)
print(f"Num IDs removed: {orig_count - new_count}")

In [ ]:
# Some rows in the example assay table contain invalid characters,
# eg. the AU_gpt column contains the string *NS* instead of a floating point number.
# In this example we replace the *NS* string with a value of *-999.0*.
# The table also contains some *less than* symbols (ie. '<') which also need to be removed.
df = df.replace(to_replace="NS", value=-999.0)
df = df.loc[df["AU_gpt"].str[:1] != "<"]

In [ ]:
# We need to create a look-up table but the hole IDs must be grouped and ordered, so we sort the dataframe accordingly.
# We also count the number of rows in the final dataframe.
df.sort_values(by=[object_hole_id], inplace=True)
num_rows = len(df.index)

In [ ]:
# Create an **assay_bhid_values** dataframe and copy the hole IDs.
assay_bhid_values_df = pd.DataFrame()
assay_bhid_values_df["data"] = df[object_hole_id]
assay_bhid_values_df.head()

### Intervals

In [ ]:
# Create a dataframe and copy the *FROM* and *TO* columns to it.
assay_interval_table_df = df.loc[:, ["FROM", "TO"]].copy().reset_index(drop=True)
assay_interval_table_df.columns = ["data1", "data2"]
assay_interval_table_df.head()

In [ ]:
# Create a component that represents the intervals.
start_and_end = FloatArray2_V1_0_1.from_dict(data_client.save_dataframe(assay_interval_table_df))

### Holes

In [ ]:
# Create a mapping between the hole IDs in the assay table and the hole IDs from the collar table.
assay_holes_df = create_hole_id_mapping(hole_id_table=hole_id_table_df, value_list=assay_bhid_values_df)

In [ ]:
# Create a component that represents the assay holes.
location_holes_component = DownholeCollection_V1_2_0_Location_Holes.from_dict(
    data_client.save_dataframe(assay_holes_df)
)

### Collection Attributes

In [ ]:
# Construct a FloatArray1 component for each column in the assay table.
attributes = []

for heading_name, heading_key in object_attributes_ref["Wolfpass_WP_assay"].items():
    values_df = pd.DataFrame()
    values_df = df.loc[:, [heading_name]].copy().astype(float).reset_index(drop=True)
    values = FloatArray1_V1_0_1.from_dict(data_client.save_dataframe(values_df))

    attribute = ContinuousAttribute_V1_1_0(
        name=heading_name, key=heading_key, nan_description=NanContinuous_V1_0_1(values=[]), values=values
    )

    attributes.append(attribute)

In [ ]:
# Create the `DownholeCollection_V1_2_0_Collections_IntervalTable` object by combining the *holes* and *from_to* components.
from_to_component = IntervalTable_V1_2_0_FromTo(
    intervals=Intervals_V1_0_1(start_and_end=start_and_end),
    attributes=attributes,
)

collection = DownholeCollection_V1_2_0_Collections_IntervalTable(
    name=collection_name, from_to=from_to_component, holes=location_holes_component
)

collections.append(collection)

### Lithology

In [ ]:
# Enter the name of the input csv file (without the file extension) in the `collection_name` variable.
# The same name will be applied to this collection in the new geoscience object, but this is configurable.
collection_name = "Wolfpass_WP_lith"
input_file_path = f"{input_path}/{collection_name}.csv"

# Load the lithology file and find the length.
df = pd.read_csv(input_file_path)
orig_count = len(df)
df.head()

In [ ]:
# Remove hole IDs from the survey file that don't appear in the hole ID table.
# If you leave these rogue hole IDs in your survey table Leapfrog won't be able to import your geoscience object.
df.drop(df[~df[object_hole_id].isin(list(hole_id_table_df["value"]))].index, inplace=True)
df.reset_index(drop=True)
new_count = len(df)
print(f"Num IDs removed: {orig_count - new_count}")

In [ ]:
# We need to create a look-up table but the hole IDs must be grouped and ordered, so we sort the dataframe accordingly.
# We also count the number of rows in the final dataframe.
df.sort_values(by=[object_hole_id], inplace=True)
num_rows = len(df.index)

In [ ]:
# Create a **lith_bhid_values** dataframe and copy the hole IDs.
lith_bhid_values_df = pd.DataFrame()
lith_bhid_values_df["data"] = df[object_hole_id]
lith_bhid_values_df.head()

### Intervals

In [ ]:
# Create a dataframe and copy the `FROM` and `TO` columns to it.
lith_interval_table_df = pd.DataFrame()
lith_interval_table_df = df.loc[:, ["FROM", "TO"]].copy().reset_index(drop=True)
lith_interval_table_df.head()

In [ ]:
# Create a component that presents the intervals.
start_and_end = FloatArray2_V1_0_1.from_dict(data_client.save_dataframe(lith_interval_table_df))

### Holes

In [ ]:
# Create a mapping between the hole IDs in the lithology table and the hole IDs from the collar table.
lith_holes_df = create_hole_id_mapping(hole_id_table=hole_id_table_df, value_list=lith_bhid_values_df)

In [ ]:
# Create a component that represents the lithology holes.
location_holes_component = DownholeCollection_V1_2_0_Location_Holes.from_dict(data_client.save_dataframe(lith_holes_df))

### Collection Attributes

In [ ]:
# Construct a FloatArray1 component for each column in the assay table.
attributes = []

for heading_name, heading_key in object_attributes_ref["Wolfpass_WP_lith"].items():
    lith_df = pd.DataFrame()
    lith_df["data"] = df[heading_name]
    lith_df = lith_df.astype(str).reset_index(drop=True)
    table_df, values_df = create_category_lookup_and_values(lith_df)

    table = LookupTable_V1_0_1.from_dict(data_client.save_dataframe(table_df))
    values = IntegerArray1_V1_0_1.from_dict(data_client.save_dataframe(values_df))

    attribute = CategoryAttribute_V1_1_0(
        name=heading_name, key=heading_key, nan_description=NanCategorical_V1_0_1(values=[]), table=table, values=values
    )
    attributes.append(attribute)

In [ ]:
# Create the `DownholeCollection_V1_2_0_Collections_IntervalTable` object by combining the *holes* and *from_to* components.
from_to_component = IntervalTable_V1_2_0_FromTo(
    intervals=Intervals_V1_0_1(
        start_and_end=start_and_end,
    ),
    attributes=attributes,
)

collection = DownholeCollection_V1_2_0_Collections_IntervalTable(
    name=collection_name, from_to=from_to_component, holes=location_holes_component
)

collections.append(collection)

### Assemble the downhole collection object

In [ ]:
# Lastly, assemble the complete geoscience object by combining all previously defined components.
# - The name and UUID are used to identify the object.
# - The UUID is set to None because this is a new object. A new UUID will be assigned by the Evo service.
# - The bounding box defines the spatial extent of the object.
# - The tags provide metadata about the object.
# - The coordinate reference system defines the spatial reference for the object.
# - The locations component contains the coordinates and attributes.
downhole_collection = DownholeCollection_V1_2_0(
    name=object_name,
    uuid=None,
    bounding_box=bounding_box,
    tags=object_tags,
    coordinate_reference_system=coordinate_reference_system,
    location=location_component,
    collections=collections,
)

await data_client.upload_referenced_data(downhole_collection.as_dict(), FeedbackWidget("Uploading data"))
new_downhole_collection_metadata = await object_client.create_geoscience_object(
    full_obj_path, downhole_collection.as_dict()
)

### View the object in the Evo portal

In [ ]:
build_portal_url(new_downhole_collection_metadata)

Success! You now have a new geoscience object in Evo containing your downhole-collection data.

## Summary

In this example, we've completed the following:
* Analysed the collar and survey tables and constructed the elements and components required for locations.
* Analysed the data columns and constructed the elements and components required for attributes.
* Converted the input location and attribute data into Parquet format and saved it to the local cache.
* Combined all of the elements, components and data references into the downhole-collection schema format.
* Uploaded the Parquet files and the newly assembled object in JSON format to Evo.